In [26]:
from flask import Flask, render_template, send_from_directory, request, Response, stream_with_context, jsonify
import subprocess
import pandas as pd

app = Flask(__name__, template_folder='C:/Users/leramero/Desktop/Portfolio', static_folder='C:/Users/leramero/Desktop/Portfolio/assets')

class PokemonCombat:
    def __init__(self):
        self.proceso = None

    def iniciar_combate(self, pokemon1, pokemon2, pk1_ability1, pk1_ability2, pk1_ability3, pk1_ability4, pk2_ability1, pk2_ability2, pk2_ability3, pk2_ability4):
        # Iniciar la ejecución del script
        script = "projects/codes/pokemon.py"
        print(pokemon1, pokemon2, pk1_ability1, pk1_ability2, pk1_ability3, pk1_ability4, pk2_ability1, pk2_ability2, pk2_ability3, pk2_ability4)
        self.proceso = subprocess.Popen(["python", "-u", script, pokemon1, pokemon2, pk1_ability1, pk1_ability2, pk1_ability3, pk1_ability4, pk2_ability1, pk2_ability2, pk2_ability3, pk2_ability4], stdout=subprocess.PIPE, universal_newlines=True)
        return Response("Iniciado", mimetype="text/event-stream", headers={
            "Cache-Control": "no-cache",
            "Content-Type": "text/event-stream"
        })

    def combat_stream(self):
        # Leer la salida del script y enviarla como eventos
        def generar_eventos():
            while True:
                salida = self.proceso.stdout.readline()
                if salida:
                    yield "data: " + salida + "\n\n"
                else:
                    self.cerrar_proceso()
                    yield "data: " + "Error!!" + "\n\n"
                    break
        return Response(generar_eventos(), mimetype="text/event-stream", headers={
            "Cache-Control": "no-cache",
            "Content-Type": "text/event-stream"
        })

    def cerrar_proceso(self):
        # Cerrar el proceso
        if self.proceso:
            self.proceso.kill()
            self.proceso = None

pokemon_combat = PokemonCombat()

@app.route("/")
def index():
    return render_template("index.html")

@app.route("/assets/<path:path>")
def send_static(path):
    return send_from_directory("assets", path)

@app.route("/projects/assets/pk/<path:path>")
def send_static_pk(path):
    return send_from_directory("projects/assets/pk", path)

@app.route("/projects/assets/<path:path>")
def send_static_pr(path):
    return send_from_directory("projects/assets", path)

@app.route("/assets/images/<path:path>")
def send_image(path):
    return send_from_directory("assets/images", path)

@app.route("/projects/assets/images/<path:path>")
def send_image_pk(path):
    return send_from_directory("projects/assets/images", path)

@app.route("/projects/<path:path>")
def pokemon(path):
    url = 'https://drive.google.com/file/d/1NulAOMTMRPDzsfh9-DBMdiiPlKQWZoEH/view?usp=sharing'
    path = 'https://drive.google.com/uc?export=download&id=' + url.split('/')[-2]
    pk1 = pd.read_csv(path)
    pk_list = pk1[~((pk1['Name'].str.contains("Mega")) | (pk1['Name'].str.contains("Gigantamax")))]['Name'].tolist()
    return render_template("projects/pokemon.html", pk_list=pk_list)

@app.route("/pokemon_combat", methods=["POST"])
def pokemon_combat_inicio():
    pokemon1 = request.json["pokemon1"]
    pk1_ability1 = request.json["pk1_ability1"]
    pk1_ability2 = request.json["pk1_ability2"]
    pk1_ability3 = request.json["pk1_ability3"]
    pk1_ability4 = request.json["pk1_ability4"]
    pokemon2 = request.json["pokemon2"]
    pk2_ability1 = request.json["pk2_ability1"]
    pk2_ability2 = request.json["pk2_ability2"]
    pk2_ability3 = request.json["pk2_ability3"]
    pk2_ability4 = request.json["pk2_ability4"]
    return pokemon_combat.iniciar_combate(pokemon1, pokemon2, pk1_ability1, pk1_ability2, pk1_ability3, pk1_ability4, pk2_ability1, pk2_ability2, pk2_ability3, pk2_ability4)

@app.route("/pokemon_combat_stream")
def pokemon_combat_stream():
    return pokemon_combat.combat_stream()

@app.route("/get_abilities1", methods=["POST"])
def get_abilities1():
    pokemon1 = request.json["pokemon1"]
    url = 'https://drive.google.com/file/d/1sbpvpGnUYM6p4tJcWT3qEqyWUCZyvDNs/view?usp=sharing'
    path = 'https://drive.google.com/uc?export=download&id=' + url.split('/')[-2]
    m_learn = pd.read_csv(path)
    m_learn = m_learn[['Pokemon', 'Move']]
    abilities = m_learn[m_learn["Pokemon"] == pokemon1]["Move"].tolist()
    return jsonify(abilities)

@app.route("/get_abilities2", methods=["POST"])
def get_abilities2():
    pokemon1 = request.json["pokemon2"]
    url = 'https://drive.google.com/file/d/1sbpvpGnUYM6p4tJcWT3qEqyWUCZyvDNs/view?usp=sharing'
    path = 'https://drive.google.com/uc?export=download&id=' + url.split('/')[-2]
    m_learn = pd.read_csv(path)
    m_learn = m_learn[['Pokemon', 'Move']]
    abilities = m_learn[m_learn["Pokemon"] == pokemon1]["Move"].tolist()
    return jsonify(abilities)

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Aug/2024 17:51:38] "GET /projects/pokemon.html HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2024 17:51:38] "GET /projects/assets/css-pokemon-gameboy.css HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2024 17:51:38] "GET /projects/assets/images/fondo_pk.jpg HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2024 17:51:38] "GET /projects/assets/images/pk.ico HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2024 17:51:43] "POST /get_abilities1 HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2024 17:51:47] "POST /get_abilities2 HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2024 17:51:48] "POST /pokemon_combat HTTP/1.1" 200 -


Venusaur Blastoise Amnesia Amnesia Amnesia Amnesia Aqua Jet Aqua Jet Aqua Jet Aqua Jet


127.0.0.1 - - [23/Aug/2024 17:51:49] "GET /pokemon_combat_stream HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2024 17:52:29] "GET /pokemon_combat_stream HTTP/1.1" 500 -
Error on request:
Traceback (most recent call last):
  File "C:\Users\leramero\Anaconda3\Lib\site-packages\werkzeug\serving.py", line 324, in run_wsgi
    execute(self.server.app)
  File "C:\Users\leramero\Anaconda3\Lib\site-packages\werkzeug\serving.py", line 315, in execute
    for data in application_iter:
  File "C:\Users\leramero\Anaconda3\Lib\site-packages\werkzeug\wsgi.py", line 462, in __next__
    return self._next()
  File "C:\Users\leramero\Anaconda3\Lib\site-packages\werkzeug\wrappers\response.py", line 50, in _iter_encoded
    for item in iterable:
  File "C:\Users\leramero\AppData\Local\Temp\ipykernel_2248\648506906.py", line 25, in generar_eventos
    
AttributeError: 'NoneType' object has no attribute 'stdout'
127.0.0.1 - - [23/Aug/2024 17:53:04] "GET /projects/pokemon.html HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug

Caterpie Pidgey Bug Bite Bug Bite Bug Bite Bug Bite Aerial Ace Aerial Ace Aerial Ace Aerial Ace


127.0.0.1 - - [23/Aug/2024 17:53:15] "GET /pokemon_combat_stream HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2024 17:53:31] "GET /pokemon_combat_stream HTTP/1.1" 500 -
Error on request:
Traceback (most recent call last):
  File "C:\Users\leramero\Anaconda3\Lib\site-packages\werkzeug\serving.py", line 324, in run_wsgi
    execute(self.server.app)
  File "C:\Users\leramero\Anaconda3\Lib\site-packages\werkzeug\serving.py", line 315, in execute
    for data in application_iter:
  File "C:\Users\leramero\Anaconda3\Lib\site-packages\werkzeug\wsgi.py", line 462, in __next__
    return self._next()
  File "C:\Users\leramero\Anaconda3\Lib\site-packages\werkzeug\wrappers\response.py", line 50, in _iter_encoded
    for item in iterable:
  File "C:\Users\leramero\AppData\Local\Temp\ipykernel_2248\648506906.py", line 25, in generar_eventos
    
AttributeError: 'NoneType' object has no attribute 'stdout'
127.0.0.1 - - [23/Aug/2024 17:53:44] "GET /projects/pokemon.html HTTP/1.1" 200 -
